In [2]:
from sqlalchemy import types, create_engine
from sqlalchemy.orm import sessionmaker
import sqlalchemy.types as sqltypes
import cx_Oracle
from msilib import Table
from operator import index
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by   import By
from selenium.webdriver.support.select  import Select
from win32com import client as win32
import pandas as pd
import json 
import csv
import re


In [3]:
cx_Oracle.init_oracle_client(lib_dir=r"C:\Oracle\client12c\instantclient_19_15")

In [4]:
navegador = webdriver.Chrome() # ABRE O NAVEGADOR
navegador.maximize_window()
navegador.get("https://extranet.portonave.com.br/login/") # ENTRA NO SITE PRE-DEFINIDO
navegador.find_element(By.ID, 'politicaDePrivacidade').send_keys(Keys.ENTER) # PRECIONA ENTER NA POLICITA DE PRIVACIDADE
login = navegador.find_element(By.ID, 'login') # BUSCA O CAMPO DE LOGIN
senha = navegador.find_element(By.ID, 'senha') # BUSCA O CAMPO DE SENHA

sleep(2)
login.send_keys('95704450091') # APLICANDO LOGIN
senha.send_keys('Trust.2022' + Keys.ENTER)  # APLICANDO SENHA E ENTRANDO


sleep(10)

select = Select(navegador.find_element(By.ID, 'empresaId')) # CAMPO DE SELEÇÃOconsolas 'courier new' monospac
select.select_by_value('32295') # SELECIONA O TTL BRAZIL - 07426908000100 - Despachante

entrar = navegador.find_element(By.XPATH, '//*[@id="app"]/div/section/div/div[2]/form/div/span/button').click() # CLICA EM ENTRAR
sleep(3)
opdc = navegador.find_element(By.XPATH, '//*[@id="app"]/div/aside/div/nav/ul/li[6]/div').click() # CLICA EM OPERAÇÃO/DOCUMENTAL 
sleep(2)
vistoria = navegador.find_element(By.XPATH, '/html/body/div[3]/div/aside/div/nav/ul/li[6]/ul/li[5]').click() # CLICA EM VISTORIAS(HISTORICO)

sleep(5)
navegador.switch_to.window(navegador.window_handles[1])

link = navegador.find_element(By.XPATH, '//*[@id="side-menu"]/li[7]').click()
sleep(2)

lista_df = []
npaginas = 2
for i in range(npaginas, 5):
    pagina = navegador.find_element(By.XPATH, f'//*[@id="content"]/form[2]/div[3]/ul/li[{npaginas}]/a')
    navegador.execute_script("arguments[0].click();", pagina)
    sleep(5)

    fluxo = navegador.find_element(By.ID, 'resultados')
    fluxo_html = fluxo.get_attribute('outerHTML')
    fluxo_dfs = pd.read_html(fluxo_html)
    df = fluxo_dfs[0]
    df = df.rename(columns={'CE E Conhecimento': 'CE', 'Datas': 'DATAS', 'Situação': 'SITUACAO'})
    df_ret = pd.DataFrame(df)

    lista_df.append(df_ret)
    npaginas = npaginas + 1

#navegador.quit()

pgn = pd.concat(lista_df).to_json(orient='records') # JUNTA TODAS AS PAGINAS

json_objt = json.loads(pgn)
lista_ce = []
lista_data = []
lista_situ = []

for c in json_objt:
    lista_ce.append(c['CE'].split()[-1])
    lista_data.append(c['DATAS'].split()[-2])
    lista_situ.append(c['SITUACAO'])
    
resultado = dict(zip(lista_ce, lista_data))
envia_email = dict(zip(lista_ce, lista_situ))

for chave, valor in envia_email.items():
        if valor == 'Selecionado para vistoria':
            html_tr = ''
            html_tr  += f''' 
            <tr style="border: 1px solid black">
                <td style="border: 1px solid black; padding: 5px; margin: 1px; text-align: center">{chave}</td>
                <td style="border: 1px solid black; padding: 5px; margin: 1px; text-align: center">{valor}</td>
            </tr>
            '''
outlook = win32.Dispatch('outlook.application') # AQUI É ENVIADO O EMAIL 
email = outlook.CreateItem(0)

email.To = 'jeremias.crispim@trust.group'
email.Subject = 'E-mail de teste'
email.HTMLBody = f"""
<p>Novo CE selecionado para vistoria encontrado!</p>
    <table  style="border: 1px solid black">
        <thead> 
            <tr>
                <th style="border: 1px solid black; background-color: #27B19E ">CE</th>
                <th style="border: 1px solid black; background-color: #27B19E ">Situação</th>
            </tr>
        </thead>
        <tbody> 
            {html_tr}
        </tbody>
    </table>   
"""

email.Send()
print('Email enviado')

dsn_tns = cx_Oracle.makedsn('192.168.1.177', '1521', service_name='CONEXOS')
conn = cx_Oracle.connect(user=r'APP_TRUST', password='TRST.1290', dsn=dsn_tns)
c = conn.cursor()

for key, value in resultado.items():
    if len(value) != 5:
        c.execute(f''' INSERT INTO LOG_CE_MASTER (FILIAL, CNT_COD, CE_PESQUISADO, DATA_ANTERIOR, QUANDO) 
                        SELECT
                            FIL_COD,
                            CNT_COD,
                            CNT_ESP_NICKMASTER,
                            CNT_DTA_LIBERACAO_MAPA,
                            SYSDATE
                        FROM
                            PRC_CONHECIMENTO_TRASP@TRUST_HLG
                        WHERE
                            CNT_ESP_NICKMASTER = '{key}' ''')
        c.execute(f'''UPDATE LOG_CE_MASTER SET DATA_INSERIDA = '{value}' WHERE CE_PESQUISADO = '{key}' ''')
        c.execute(f''' update prc_conhecimento_trasp@TRUST_HLG set CNT_DTA_LIBERACAO_MAPA = '{value}' where cnt_esp_nickmaster = '{key}' ''')
        conn.commit()
c.close()

Email enviado


In [34]:
teste = '''="INSERT INTO COM_DOC_PRODTRIB (FIL_COD, DOC_TIP, DOC_COD, FIS_COD, PRD_COD, DPR_COD_SEQ, IMP_COD, USN_COD, DTR_NUM_ORDEM, DTR_VLD_ACAO_TOTAL, DTR_VLD_ACAO_CUSTO,
DTR_NUM_INCIDENCIA, DTR_VLD_ACUMULADO, DTR_VLD_FORMA_RATEIO, DTR_VLD_PLANILHA_CUSTO) VALUES
("&A2&", '1', "&B2&", '1', "&F2&", "&E2&", '1150', '5', '202', '0', '0', '0', '0', '0', '0') "'''


In [36]:
len(teste)

349

In [ ]:
for c in json_objt:
    lista_ce.append(c['CE'].split()[-1])
    lista_data.append(c['DATAS'].split()[-2])
    lista_situ.append(c['SITUACAO'])
    
resultado = dict(zip(lista_ce, lista_data))
envia_email = dict(zip(lista_ce, lista_situ))

for chave, valor in envia_email.items():
        if valor == 'Selecionado para vistoria':
            html_tr = ''
            html_tr  += f''' 
            <tr style="border: 1px solid black">
                <td style="border: 1px solid black; padding: 5px; margin: 1px; text-align: center">{chave}</td>
                <td style="border: 1px solid black; padding: 5px; margin: 1px; text-align: center">{valor}</td>
            </tr>
            '''
outlook = win32.Dispatch('outlook.application') # AQUI É ENVIADO O EMAIL 
email = outlook.CreateItem(0)

email.To = 'jeremias.crispim@trust.group'
email.Subject = 'E-mail de teste'
email.HTMLBody = f"""
<p>Novo CE selecionado para vistoria encontrado!</p>
    <table  style="border: 1px solid black">
        <thead> 
            <tr>
                <th style="border: 1px solid black; background-color: #27B19E ">CE</th>
                <th style="border: 1px solid black; background-color: #27B19E ">Situação</th>
            </tr>
        </thead>
        <tbody> 
            {html_tr}
        </tbody>
    </table>   
"""

email.Send()
print('Email enviado')

In [5]:
html_tr

' \n            <tr style="border: 1px solid black">\n                <td style="border: 1px solid black; padding: 5px; margin: 1px; text-align: center">182205110212685</td>\n                <td style="border: 1px solid black; padding: 5px; margin: 1px; text-align: center">Selecionado para vistoria</td>\n            </tr>\n            '

In [ ]:
pgn = pd.concat(lista_df).to_json(orient='records') # JUNTA TODAS AS PAGINAS

json_objt = json.loads(pgn)
lista_ce = []
lista_data = []
lista_situ = []

for c in json_objt:
    lista_ce.append(c['CE'].split()[-1])
    lista_data.append(c['DATAS'].split()[-2])
    lista_situ.append(c['SITUACAO'])
    
resultado = dict(zip(lista_ce, lista_data))
envia_email = dict(zip(lista_ce, lista_situ))

In [37]:
resultado.items()

dict_items([('182205140251751', '16:29'), ('182205139301686', '16:25'), ('182205139164670', '16:22'), ('182205130016806', '16:00'), ('182205138579915', '13/06/2022'), ('182205135998011', '13/06/2022'), ('182205136095643', '10/06/2022'), ('182205136095562', '10/06/2022'), ('182205130665871', '09/06/2022'), ('182205130292234', '06/06/2022'), ('182205130293559', '06/06/2022'), ('182205132254216', '03/06/2022'), ('182205132558193', '03/06/2022'), ('182205132052508', '03/06/2022'), ('182205132052699', '03/06/2022'), ('182205130311204', '01/06/2022'), ('182205125959420', '01/06/2022'), ('182205130016474', '01/06/2022'), ('182205123312527', '01/06/2022'), ('182205127249803', '31/05/2022'), ('182205127954108', '31/05/2022'), ('182205126831592', '31/05/2022'), ('182205102896317', '26/05/2022'), ('182205102896236', '26/05/2022'), ('182205123856462', '26/05/2022'), ('182205123313256', '26/05/2022'), ('182205123429475', '26/05/2022'), ('182205123470785', '26/05/2022'), ('182205122318009', '26/05/2

In [ ]:
if lista_situ == 'Selecionado para vistoria':
    html_tr = ''
    for chave, valor in envia_email.items():# AQUI A MAGICA ACONTECE, PARA CADA LINHA DO JSON É RETIRADO O CE E A SITUAÇÃO PARA ENTRAR NO TD DO HTML
        html_tr  += f''' 
    <tr style="border: 1px solid black">
        <td style="border: 1px solid black; padding: 5px; margin: 1px; text-align: center">{chave}</td>
        <td style="border: 1px solid black; padding: 5px; margin: 1px; text-align: center">{valor}</td>
    </tr>
    '''

In [19]:
lista_situ == 'Selecionado para vistoria'

False

In [ ]:
pgn = pd.concat(lista_df).to_json(orient='records') # JUNTA TODAS AS PAGINAS

json_objt = json.loads(pgn)
lista_ce = []
lista_data = []
lista_situ = []

for c in json_objt:
    lista_ce.append(c['CE'].split()[-1])
    lista_data.append(c['DATAS'].split()[-2])
    lista_situ.append(c['SITUACAO'])
    
    if lista_situ == 'Selecionado para vistoria':
        html_tr = ''
        for chave, valor in envia_email.items():# AQUI A MAGICA ACONTECE, PARA CADA LINHA DO JSON É RETIRADO O CE E A SITUAÇÃO PARA ENTRAR NO TD DO HTML
            html_tr  += f''' 
            <tr style="border: 1px solid black">
                <td style="border: 1px solid black; padding: 5px; margin: 1px; text-align: center">{chave}</td>
                <td style="border: 1px solid black; padding: 5px; margin: 1px; text-align: center">{valor}</td>
            </tr>
            '''
            outlook = win32.Dispatch('outlook.application') # AQUI É ENVIADO O EMAIL 
            email = outlook.CreateItem(0)

            email.To = 'jeremias.crispim@trust.group'
            email.Subject = 'E-mail de teste'
            email.HTMLBody = f"""
            <p>Novo CE selecionado para vistoria encontrado!</p>
                <table  style="border: 1px solid black">
                    <thead> 
                        <tr>
                            <th style="border: 1px solid black; background-color: #27B19E ">CE</th>
                            <th style="border: 1px solid black; background-color: #27B19E ">Situação</th>
                        </tr>
                    </thead>
                    <tbody> 
                        {html_tr}
                    </tbody>
                </table>   
            """

            email.Send()
            print('Email enviado')
        
        
resultado = dict(zip(lista_ce, lista_data))
envia_email = dict(zip(lista_ce, lista_situ))

In [ ]:
json_objt = json.loads(teste)
lista_ce = []
lista_data = []
lista_situ = []

for c in json_objt:
    lista_ce.append(c['CE'].split()[-1])
    lista_data.append(c['DATAS'].split()[-2])
    if c['SITUACAO'] == 'Selecionado para vistoria':
        lista_situ.append(c['SITUACAO'])
        html_tr = ''
        for chave, valor in envia_email.items():# AQUI A MAGICA ACONTECE, PARA CADA LINHA DO JSON É RETIRADO O CE E A SITUAÇÃO PARA ENTRAR NO TD DO HTML
            html_tr  += f''' 
        <tr style="border: 1px solid black">
            <td style="border: 1px solid black; padding: 5px; margin: 1px; text-align: center">{chave}</td>
            <td style="border: 1px solid black; padding: 5px; margin: 1px; text-align: center">{valor}</td>
        </tr>
        '''
        
resultado = dict(zip(lista_ce, lista_data))
envia_email = dict(zip(lista_ce, lista_situ))




outlook = win32.Dispatch('outlook.application') # AQUI É ENVIADO O EMAIL 
email = outlook.CreateItem(0)

email.To = 'jeremias.crispim@trust.group'
email.Subject = 'E-mail de teste'
email.HTMLBody = f"""
<p>Novo CE selecionado para vistoria encontrado!</p>
    <table  style="border: 1px solid black">
        <thead> 
            <tr>
                <th style="border: 1px solid black; background-color: #27B19E ">CE</th>
                <th style="border: 1px solid black; background-color: #27B19E ">Situação</th>
            </tr>
        </thead>
        <tbody> 
            {html_tr}
        </tbody>
    </table>   
"""

email.Send()
print('Email enviado')

In [ ]:
lista_situ = []
for c in json_objt:
    if c['SITUACAO'] == 'Selecionado para vistoria':
        lista_situ.append(c['SITUACAO'])

In [ ]:
envia_email = dict(zip(lista_ce, lista_situ))

In [ ]:
envia_email

In [ ]:
html_tr = ''
for chave, valor in envia_email.items():# AQUI A MAGICA ACONTECE, PARA CADA LINHA DO JSON É RETIRADO O CE E A SITUAÇÃO PARA ENTRAR NO TD DO HTML
    html_tr  += f''' 
<tr style="border: 1px solid black">
    <td style="border: 1px solid black; padding: 5px; margin: 1px; text-align: center">{chave}</td>
    <td style="border: 1px solid black; padding: 5px; margin: 1px; text-align: center">{valor}</td>
</tr>
'''

In [ ]:
html_tr

In [ ]:
outlook = win32.Dispatch('outlook.application') # AQUI É ENVIADO O EMAIL 
email = outlook.CreateItem(0)

email.To = 'jeremias.crispim@trust.group'
email.Subject = 'E-mail de teste'
email.HTMLBody = f"""
<p>Novo CE selecionado para vistoria encontrado!</p>
    <table  style="border: 1px solid black">
        <thead> 
            <tr>
                <th style="border: 1px solid black; background-color: #27B19E ">CE</th>
                <th style="border: 1px solid black; background-color: #27B19E ">Situação</th>
            </tr>
        </thead>
        <tbody> 
            {html_tr}
        </tbody>
    </table>   
"""

email.Send()
print('Email enviado')

In [95]:
dsn_tns = cx_Oracle.makedsn('192.168.1.229', '1521', service_name='CONEXOS')
conn = cx_Oracle.connect(user=r'TRUST', password='JDSD7723B4WEFR', dsn=dsn_tns)
c = conn.cursor()

In [7]:
dsn_tns = cx_Oracle.makedsn('192.168.1.177', '1521', service_name='CONEXOS')
conn = cx_Oracle.connect(user=r'APP_TRUST', password='TRST.1290', dsn=dsn_tns)
c = conn.cursor()

In [135]:
for column in c.execute(" update log_ce_master set cnt_cod = '23745' "):
    print(column)

TypeError: 'NoneType' object is not iterable

In [8]:
for column in c.execute("select * from LOG_CE_MASTER where rownum <= 10"):
    print(column)

(1, '23744', '182205099962067', None, None, datetime.datetime(2022, 6, 10, 16, 9, 13))


In [17]:
resultado.items()

dict_items([('182205140251751', '16:29'), ('182205139301686', '16:25'), ('182205139164670', '16:22'), ('182205130016806', '16:00'), ('182205138579915', '13/06/2022'), ('182205135998011', '13/06/2022'), ('182205136095643', '10/06/2022'), ('182205136095562', '10/06/2022'), ('182205130665871', '09/06/2022'), ('182205130292234', '06/06/2022'), ('182205130293559', '06/06/2022'), ('182205132254216', '03/06/2022'), ('182205132558193', '03/06/2022'), ('182205132052508', '03/06/2022'), ('182205132052699', '03/06/2022'), ('182205130311204', '01/06/2022'), ('182205125959420', '01/06/2022'), ('182205130016474', '01/06/2022'), ('182205123312527', '01/06/2022'), ('182205127249803', '31/05/2022'), ('182205127954108', '31/05/2022'), ('182205126831592', '31/05/2022'), ('182205102896317', '26/05/2022'), ('182205102896236', '26/05/2022'), ('182205123856462', '26/05/2022'), ('182205123313256', '26/05/2022'), ('182205123429475', '26/05/2022'), ('182205123470785', '26/05/2022'), ('182205122318009', '26/05/2

In [ ]:
dsn_tns = cx_Oracle.makedsn('192.168.1.177', '1521', service_name='CONEXOS')
conn = cx_Oracle.connect(user=r'APP_TRUST', password='TRST.1290', dsn=dsn_tns)
c = conn.cursor()
for key, value in resultado.items():
    if len(value) != 5:
        c.execute(f''' INSERT INTO LOG_CE_MASTER (FILIAL, CNT_COD, CE_PESQUISADO, DATA_ANTERIOR, QUANDO) 
                        SELECT
                            FIL_COD,
                            CNT_COD,
                            CNT_ESP_NICKMASTER,
                            CNT_DTA_LIBERACAO_MAPA,
                            SYSDATE
                        FROM
                            PRC_CONHECIMENTO_TRASP@TRUST_HLG
                        WHERE
                            CNT_ESP_NICKMASTER = '{key}' ''')
        c.execute(f'''UPDATE LOG_CE_MASTER SET DATA_INSERIDA = '{value}' WHERE CE_PESQUISADO = '{key}' ''')
        c.execute(f''' update prc_conhecimento_trasp@TRUST_HLG set CNT_DTA_LIBERACAO_MAPA = '{value}' where cnt_esp_nickmaster = '{key}' ''')
        conn.commit()
c.close()